<a href="https://colab.research.google.com/github/navicester/first-order-model/blob/master/old_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demo for paper "First Order Motion Model for Image Animation"

In [1]:
import os

COLAB = False
try:  
    from google.colab import drive
    # drive.mount('/content/gdrive')
    COLAB = True
except:
    pass

if COLAB:
    BASE_DIR = os.getcwd()
    BASE_REPO_DIR = os.getcwd()
    REPO_DIR = os.path.join(BASE_REPO_DIR,"first-order-model")
    MODEL_DIR = "/content/gdrive/My Drive/first-order-motion-model"
    OUTPUT_DIR = os.getcwd()
else:
    BASE_DIR = "/binhe/"
    BASE_REPO_DIR = os.path.join(BASE_DIR,"repos")
    REPO_DIR = os.path.join(BASE_REPO_DIR,"first-order-model")
    MODEL_DIR = os.path.join(BASE_DIR,"models","first-order-model")
    OUTPUT_DIR = os.path.join(BASE_DIR,"output",'avatar')

    %cd {BASE_REPO_DIR}

/binhe/repos


**Clone repository**

In [2]:
if not os.path.exists("first-order-model"):
    !git clone https://github.com/AliaksandrSiarohin/first-order-model

In [3]:
cd first-order-model

/binhe/repos/first-order-model


**Mount your Google drive folder on Colab**

In [7]:
try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    DATA_DIR = "/content/gdrive/My Drive/first-order-motion-model"
except:
    # 这个数据集用于新的demo脚本
#     if not os.path.exists("demo"):
#         !git clone https://github.com/graphemecluster/first-order-model-demo demo
    DATA_DIR = f"{REPO_DIR}/demo_old"

**Add folder https://drive.google.com/drive/folders/1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw?usp=sharing  to your google drive.
Alternativelly you can use this mirror link https://drive.google.com/drive/folders/16inDpBRPT1UC0YMGMX3dKvRnOUsf5Dhn?usp=sharing**

In [13]:
# !pip install gdown

# import gdown

# URL of the shared Google Drive folder
# folder_url = 'https://drive.google.com/drive/folders/16inDpBRPT1UC0YMGMX3dKvRnOUsf5Dhn?usp=sharing'

# Use gdown to download files from the shared folder
# gdown.download(folder_url, output='./downloaded_files2.zip', quiet=False)


Downloading...
From: https://drive.google.com/drive/folders/16inDpBRPT1UC0YMGMX3dKvRnOUsf5Dhn?usp=sharing
To: /content/first-order-model/downloaded_files2.zip
620kB [00:00, 1.02MB/s]


'./downloaded_files2.zip'

上面代码不生效

首先将上面共享目录与自己共享

然后将它添加快捷方式

![image](https://github.com/navicester/ml-ex/assets/16224205/d52ee623-ddbd-40e1-bb3c-328a8f7d4f30)


Add driver shortcut to `/content/gdrive/My Drive/first-order-motion-model`

![image](https://github.com/navicester/ml-ex/assets/16224205/eae1d80f-9a93-4081-b6fe-96701e1c13d6)


**Load driving video and source image**

In [8]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

source_image = imageio.imread(f'{DATA_DIR}/02.png')
reader = imageio.get_reader(f'{DATA_DIR}/04.mp4')


#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]

fps = reader.get_meta_data()['fps']
driving_video = []
try:
    for im in reader:
        driving_video.append(im)
except RuntimeError:
    pass
reader.close()

driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani


HTML(display(source_image, driving_video).to_html5_video())

**Create a model and load checkpoints**

In [17]:
!pip install ffmpeg-python

In [9]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml',
                            checkpoint_path=f'{MODEL_DIR}/vox-cpk.pth.tar')

**Perform image animation**

In [10]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

#save resulting video
imageio.mimsave(f'{OUTPUT_DIR}/generated.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)
#video can be downloaded from /content folder

HTML(display(source_image, driving_video, predictions).to_html5_video())

  0%|          | 0/211 [00:00<?, ?it/s]

**In the cell above we use relative keypoint displacement to animate the objects. We can use absolute coordinates instead,  but in this way all the object proporions will be inherited from the driving video. For example Putin haircut will be extended to match Trump haircut.**

In [11]:
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=False, adapt_movement_scale=True)
HTML(display(source_image, driving_video, predictions).to_html5_video())

  0%|          | 0/211 [00:00<?, ?it/s]

## Running on your data

**First we need to crop a face from both source image and video, while simple graphic editor like paint can be used for cropping from image. Cropping from video is more complicated. You can use ffpmeg for this.**

In [32]:
!ffmpeg -i /content/gdrive/My\ Drive/first-order-motion-model/07.mkv -ss 00:08:57.50 -t 00:00:08 -filter:v "crop=600:600:760:50" -async 1 hinton.mp4

ffmpeg version 2.8.17-0ubuntu0.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 5.4.0 (Ubuntu 5.4.0-6ubuntu1~16.04.12) 20160609
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --build-suffix=-ffmpeg --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --cc=cc --cxx=g++ --enable-gpl --enable-shared --disable-stripping --disable-decoder=libopenjpeg --disable-decoder=libschroedinger --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmodplug --enable-libmp3lame --enable-libopenjpeg --enable-libopus --enable-libpulse --enable-librtmp --enable-libschroedinger --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvo

**Another posibility is to use some screen recording tool, or if you need to crop many images at ones use face detector(https://github.com/1adrianb/face-alignment) , see https://github.com/AliaksandrSiarohin/video-preprocessing for preprcessing of VoxCeleb.**

In [13]:
source_image = imageio.imread(f'{DATA_DIR}/09.png')
# driving_video = imageio.mimread('hinton.mp4', memtest=False)
driving_video = imageio.mimread(f'{DATA_DIR}/00.mp4', memtest=False)


#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True,
                             adapt_movement_scale=True)

HTML(display(source_image, driving_video, predictions).to_html5_video())

  0%|          | 0/265 [00:00<?, ?it/s]